In [12]:
import pickle
import time
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score

In [13]:
n_epoch = 40
node_dim = 64
num_channels = 2
learning_rate = 0.005
weight_decay = 0.001
num_layers = 2
flag_norm = True
adaptive_lr = True
dataset_name = 'DBLP'

In [14]:
with open('data/'+dataset_name+'/node_features.pkl','rb') as f:
    node_features = pickle.load(f)
with open('data/'+dataset_name+'/edges.pkl','rb') as f:
    edges = pickle.load(f)
with open('data/'+dataset_name+'/labels.pkl','rb') as f:
    labels = pickle.load(f)
num_nodes = edges[0].shape[0]

In [15]:
node_features = torch.from_numpy(node_features).type(torch.FloatTensor)

In [16]:
node_features.shape

torch.Size([18405, 334])

In [17]:
node_features[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [18]:
edges

[<18405x18405 sparse matrix of type '<class 'numpy.int64'>'
 	with 19645 stored elements in Compressed Sparse Row format>,
 <18405x18405 sparse matrix of type '<class 'numpy.int64'>'
 	with 19645 stored elements in Compressed Sparse Column format>,
 <18405x18405 sparse matrix of type '<class 'numpy.int64'>'
 	with 14328 stored elements in Compressed Sparse Row format>,
 <18405x18405 sparse matrix of type '<class 'numpy.int64'>'
 	with 14328 stored elements in Compressed Sparse Column format>]

In [19]:
labels

[[[0, 1],
  [1, 3],
  [2, 0],
  [3, 3],
  [4, 0],
  [5, 0],
  [6, 3],
  [7, 3],
  [8, 2],
  [9, 0],
  [10, 2],
  [11, 0],
  [12, 0],
  [13, 0],
  [14, 1],
  [15, 2],
  [16, 3],
  [17, 0],
  [18, 2],
  [19, 3],
  [20, 0],
  [21, 1],
  [22, 2],
  [23, 2],
  [24, 2],
  [25, 2],
  [26, 2],
  [27, 2],
  [28, 0],
  [29, 1],
  [30, 0],
  [31, 0],
  [32, 1],
  [33, 2],
  [34, 2],
  [35, 3],
  [36, 3],
  [37, 2],
  [38, 3],
  [39, 0],
  [40, 3],
  [41, 2],
  [42, 1],
  [43, 2],
  [44, 1],
  [45, 0],
  [46, 0],
  [47, 1],
  [48, 0],
  [49, 1],
  [50, 1],
  [51, 2],
  [52, 3],
  [53, 3],
  [54, 2],
  [55, 2],
  [56, 3],
  [57, 1],
  [58, 3],
  [59, 2],
  [60, 0],
  [61, 1],
  [62, 1],
  [63, 3],
  [64, 3],
  [65, 2],
  [66, 2],
  [67, 2],
  [68, 1],
  [69, 1],
  [70, 2],
  [71, 3],
  [72, 0],
  [73, 0],
  [74, 0],
  [75, 1],
  [76, 0],
  [77, 3],
  [78, 0],
  [79, 0],
  [80, 1],
  [81, 0],
  [82, 1],
  [83, 2],
  [84, 2],
  [85, 3],
  [86, 0],
  [87, 3],
  [88, 0],
  [89, 0],
  [90, 2],
  [91, 3]

In [21]:
train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.LongTensor)
train_label = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.LongTensor)
valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.LongTensor)
valid_label = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.LongTensor)
test_node = torch.from_numpy(np.array(labels[2])[:,0]).type(torch.LongTensor)
test_label = torch.from_numpy(np.array(labels[2])[:,1]).type(torch.LongTensor)

In [5]:
for i, edge in enumerate(edges):
    if i == 0:
        A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
    else:
        A = torch.cat([A, torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim = -1)
A = torch.cat([A, torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim = -1)

In [6]:
num_classes = int(torch.max(train_label).item()+1)
num_edges = len(edges) + 1
A = A.unsqueeze(0).permute(0, 3, 1, 2)

In [10]:
A.shape

torch.Size([1, 5, 8994, 8994])

In [14]:
class GTN(nn.Module):
    def __init__(self, num_edges, num_channels, num_layers,
                 input_shape, node_dim, output_shape, flag_norm):
        super(GTN, self).__init__()
        self.num_edges = num_edges
        self.num_channels = num_channels
        self.num_layers = num_layers
        self.input_shape = input_shape
        self.node_dim = node_dim
        self.output_shape = output_shape
        self.flag_norm = flag_norm
        layers = []
        for i in range(num_layers):
            layers.append(GTLayer(num_edges, num_channels, flag_first=(i == 0)))
        self.layers = nn.ModuleList(layers)

        self.weight = nn.Parameter(torch.Tensor(input_shape, node_dim))

        self.linear1 = nn.Linear(node_dim * num_channels, node_dim)
        self.linear2 = nn.Linear(node_dim, output_shape)

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def normalization(self, H):
        for i in range(self.num_channels):
            if i == 0:
                retH = self.norm(H[i, :, :]).unsqueeze(0)
            else:
                retH = torch.cat((retH, self.norm(H[i, :, :]).unsqueeze(0)), dim=0)
        return retH

    def norm(self, H):
        if not self.flag_norm:
            return H
        H = H.t()
        H = H * ((torch.eye(H.shape[0]) == 0).type(torch.FloatTensor)) \
            + torch.eye(H.shape[0]).type(torch.FloatTensor)
        D = torch.sum(H, dim=1)
        D_inv = D.pow(-1)
        D_inv[D_inv == float('inf')] = 0
        D_inv = D_inv * torch.eye(H.shape[0]).type(torch.FloatTensor)
        H = torch.mm(D_inv, H)
        H = H.t()
        return H

    def conv(self, X, H):
        X = torch.mm(X, self.weight)
        H = self.norm(H)
        return torch.mm(H.t(), X)

    def forward(self, A, features, nodes):
        Ws = []
        for i in range(self.num_layers):
            if i == 0:
                H, W = self.layers[i](A)
            else:
                H = self.normalization(H)
                H, W = self.layers[i](A, H)
            Ws.append(W)

        for i in range(self.num_channels):
            if i == 0:
                X = nn.functional.relu(self.conv(features, H[i]))
            else:
                tX = nn.functional.relu(self.conv(features, H[i]))
                X = torch.cat((X, tX), dim=1)
        Y = self.linear1(X)
        Y = nn.functional.relu(Y)
        predict = self.linear2(Y[nodes])
        return predict, Ws


In [8]:
class GTLayer(nn.Module):
    def __init__(self, num_edges, num_channels, flag_first):
        super(GTLayer, self).__init__()
        self.num_edges = num_edges
        self.num_channels = num_channels
        self.flag_first = flag_first
        self.W1 = nn.Parameter(torch.Tensor(num_channels, num_edges, 1, 1))
        if flag_first:
            self.W2 = nn.Parameter(torch.Tensor(num_channels, num_edges, 1, 1))

    def reset_parameters(self):
        nn.init.constant_(self.W1, 0.1)
        if self.flag_first:
            nn.init.constant_(self.W2, 0.1)

    def forward(self, A, _H=None):
        if self.flag_first:
            _W1 = nn.functional.softmax(self.W1, dim=1).detach()
            _W2 = nn.functional.softmax(self.W2, dim=1).detach()
            Q1 = torch.sum(A * _W1, dim=1)
            Q2 = torch.sum(A * _W2, dim=1)
            A_ = torch.bmm(Q1, Q2)
            W = [_W1, _W2]
        else:
            _W1 = nn.functional.softmax(self.W1, dim=1).detach()
            Q1 = torch.sum(A * _W1, dim=1)
            A_ = torch.bmm(_H, Q1)
            W = [_W1]
        return A_, W


In [9]:
model = GTN(num_edges, num_channels, num_layers, node_features.shape[1], node_dim, num_classes, flag_norm)
if adaptive_lr == True:
    optimizer = torch.optim.Adam([{'params': model.weight},
                                  {'params': model.linear1.parameters()},
                                  {'params': model.linear2.parameters()},
                                  {'params': model.layers.parameters(), "lr":0.5}], lr=learning_rate, weight_decay=weight_decay)
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [18]:
def train(model, train_node, train_label, optimizer, criterion):
    model.zero_grad()
    model.train()
    predict, _ = model(A, node_features, train_node)
    loss = criterion(predict, train_label)
    loss.backward()
    optimizer.step()
    predict = torch.argmax(predict, dim=1)
    acc = (predict == train_label).sum().item() / train_label.numel()
    F_score = f1_score(predict.detach(), train_label, average='macro').item()
    return loss, acc, F_score


In [11]:
def test(model, test_node, test_label, criterion):
    model.eval()
    with torch.no_grad():
        predict, _ = model(A, node_features, test_node)
        loss = criterion(predict, test_label)
        predict = torch.argmax(predict, dim=1)
        acc = (predict == test_label).sum().item() / test_label.numel()
        F_score = f1_score(predict.detach(), test_label, average='macro').item()
    return loss, acc, F_score


In [19]:

best_f1 = 0.0

for epoch in range(n_epoch):
    starttime = time.time()

    for param_group in optimizer.param_groups:
        if param_group['lr'] > learning_rate:
            param_group['lr'] = param_group['lr'] * 0.9

    train_loss, train_acc, train_f_score = \
        train(model, train_node, train_label, optimizer, criterion)
    valid_loss, valid_acc, valid_f_score = \
        test(model, valid_node, valid_label, criterion)
    test_loss, test_acc, test_f_score = \
        test(model, test_node, test_label, criterion)

    endtime = time.time()
    epoch_mins = int((endtime - starttime) / 60)
    epoch_secs = int(endtime - starttime - epoch_mins * 60)

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\t Train Loss: {train_loss:.5f} | Train Acc: {train_acc * 100:.3f}% '
          f'| Train F score: {train_f_score:.3f}')
    print(f'\t Validation Loss: {valid_loss:.5f} | Validation Acc: {valid_acc * 100:.3f}% '
          f'| Validation F score: {valid_f_score:.3f}')
    print(f'\t Test Loss: {test_loss:.5f} | Test Acc: {test_acc * 100:.3f}% '
          f'| Test F score: {test_f_score:.3f}')
    with open("log.txt", "a") as f:
        f.write(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n')
        f.write(f'\t Train Loss: {train_loss:.5f} | Train Acc: {train_acc * 100:.3f}% '
                f'| Train F score: {train_f_score:.3f}\n')
        f.write(f'\t Validation Loss: {valid_loss:.5f} | Validation Acc: {valid_acc * 100:.3f}% '
                f'| Validation F score: {valid_f_score:.3f}\n')
        f.write(f'\t Test Loss: {test_loss:.5f} | Test Acc: {test_acc * 100:.3f}% '
                f'| Test F score: {test_f_score:.3f}\n')

    if best_f1 < valid_f_score:
        best_f1 = valid_f_score
        print(test_f_score)
        torch.save(model.state_dict(), 'model.pt')


tensor([[[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.2400, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.2000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0800, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0800, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0800]],

        [[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.2400, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.2000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0800, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0800, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0800]]],
       grad_fn=<BmmBackward>)
tensor([[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2400, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.200

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 323568144 bytes. Buy new RAM!
